In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn import svm

We'll try svm-tfidf on the dataset.

Here, we start by importing the datasets (csv or tsv format) we plan to use. Here I use a manualy fused dataset cause I'm the only morron not able to find a way to add the value of one column to another.

In [2]:
#rap_dataset IS NOT GOOD, no actual responces

#data=pd.read_csv('data/rap_dataset.tsv', sep='\t') #Use this one for TSV format
data=pd.read_csv('data/bible_abel_fused_content.csv') #This one CSV format

#data
data.head() #print the head (5first line) of the dataset to look at it

,Answer (English),Contenu
0,Me it reminds me of a night butterfly with th...,(A)
1,"A person on each side (inevitably, it is symme...",H
2,A saucepan on a gas from… camping. The gas sto...,Obj
3,NaN,NaN
4,NaN,NaN


We now check the dataset (important if you don't know it already, here I just check the columns names because we will select a couple of them)

In [3]:
print(data.columns) #print the columns

Index(['Answer (English)', 'Contenu'], dtype='object')


Now we choose the column we will use. Here I use Contenu, but you can replace it by Localisation for exemple if you use the global dataset. Here well we don't really need the Preparation of the dataset, just to drop the NaN variables.

In [4]:
##Test Contenu

#Preparing the dataset
df=data[["Answer (English)","Contenu"]] #we select the 2 columns we want.
df1=df.dropna()#we drop the line with NaN in it. If we don't, the model won't work.

text=np.array(df1['Answer (English)']) #we create a NumPy array with the text/answer
labels=np.array(df1['Contenu']) #other array for the coding of this answer


Splitting the dataset (train/test size)

here I split the dataset in test and train variables. You can change the "test_size" variable to change the ratio of training and testing data. 0.1 mean the test size is 10% of the total dataset.

In [50]:
x_train,x_test,y_train,y_test=train_test_split(text,labels,test_size=0.01,random_state=11) #Change test_size

Vectorizing (can modify parameters for TfidfVectorizer)

This is the hard part. Vectorisation is, well we turn the text into numbers. We have "df" things, I don't remember what they do. I think they make sure useless but omnipresent words like "a", "the"... are a bit ignored. Ngram range is the range of Ngrams we take. Here, it's unigram to trigrams (on word Ngram to 3 word Ngram)

In [51]:
tf_vectorizer=TfidfVectorizer(min_df=2,max_df=0.3,ngram_range=(1,3)) #change parameter
x_train_tfidf=tf_vectorizer.fit_transform(x_train)
x_test_tfidf=tf_vectorizer.transform(x_test)

Training the data and getting test score

Now you just run this cell. It train and give you the score

In [52]:
clf = svm.LinearSVC(dual=True, max_iter=1000)
clf.fit(x_train_tfidf, y_train)

clf.score(x_test_tfidf,y_test)

1.0

Def classify func for predicting a text

Here is testing. First cell define a method to test a string and give you the prediction of the model

second cell you call the method. Just change the string to what you want

In [53]:
def classify(article_text):
    global clf
    global tf_vectorizer
    global data
    article=tf_vectorizer.transform([article_text])
    
    return clf.predict(article)[0]


In [58]:
classify("an eye") #change string here

'(H)'

#### Small try : Finding the best test_size

random state 1 : 0.08 give 0.16

random state 2 : 0.05 give 0.25 (same for random state 4)

random state 8 : 0.03 and 0.04 give 0.33 (same 11)

random state 11 : 0.01 give 1.0 (overfitting), 0.02 and 0.05 give 0.5



In [49]:
ts=0.01
results=[]

while ts<0.2:
    x_train,x_test,y_train,y_test=train_test_split(text,labels,test_size=ts, random_state=11)

    tf_vectorizer=TfidfVectorizer(min_df=2,max_df=0.3,ngram_range=(1,3))
    x_train_tfidf=tf_vectorizer.fit_transform(x_train)
    x_test_tfidf=tf_vectorizer.transform(x_test)

    clf = svm.LinearSVC(dual=True, max_iter=1000)
    clf.fit(x_train_tfidf, y_train)

    a=clf.score(x_test_tfidf,y_test)
    print(ts,a)  
    results.append(a)  
    ts=ts+0.01

0.01 1.0
0.02 0.5
0.03 0.3333333333333333
0.04 0.3333333333333333
0.05 0.5
0.060000000000000005 0.2
0.07 0.2
0.08 0.16666666666666666
0.09 0.14285714285714285
0.09999999999999999 0.14285714285714285
0.10999999999999999 0.125
0.11999999999999998 0.1111111111111111
0.12999999999999998 0.1111111111111111
0.13999999999999999 0.1
0.15 0.09090909090909091
0.16 0.09090909090909091
0.17 0.16666666666666666
0.18000000000000002 0.15384615384615385
0.19000000000000003 0.15384615384615385
